<a href="https://colab.research.google.com/github/jetsonmom/Attendance/blob/main/perplexity_yolov8%26datacrolling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

경로가 중요합니다. 경로 설명하기
file을 누르면 content등이 있습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


!는 코랩의 ipython에서 사용하는 기호입니다.
yolov8을 만든 ultralytics를 설치해줍니다.

In [ ]:
!pip install ultralytics

필요한 라이브러리 임포트:

In [ ]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
import urllib.request


사전 훈련된 YOLOv8 모델 다운로드
content 밑에 yolov8.pt파일이 생겼습니다.
그러나 사람과 차만 인식합니다.
predefined모델을 좀 더 큰 사이즈로 바꿉니다.
:


In [ ]:
model = YOLO('yolov8n.pt')

인터넷에서 차도 사진 가져오기:
원하는 차도 사진 URL로 변경



In [ ]:
url = 'https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAxNzAxMjhfMjkx%2FMDAxNDg1NjEwNDkzNzg5.fxI75QwVJEc7awSh62miUXjfmlneYGstlAesJu_4-esg.l_WJd8JYm6r0MUz2vT1duyWQvwURPTmMuglVtH30g7Ig.JPEG.coollawyer72%2F2017012801002076100095391.jpg&type=sc960_832'

In [ ]:
urllib.request.urlretrieve(url, 'road_image.jpg')

이미지에서 객체 감지 수행
:results = model('road_image.jpg')
결과 시각화:
img = cv2.imread('road_image.jpg')
for r in results:
    annotated_img = r.plot()
cv2_imshow(annotated_img)
YOLOv8은 COCO 데이터셋으로 사전 훈련되어 있어 80개의 다양한 클래스(사람, 자동차, 신호등 등 포함)를 감지할 수 있습니다. 따라서 별도의 훈련 없이도 이러한 객체들을 인식할 수 있습니다


In [ ]:
results = model('road_image.jpg')
#결과 시각화:
img = cv2.imread('road_image.jpg')
for r in results:
    annotated_img = r.plot()
cv2_imshow(annotated_img)

In [ ]:
model = YOLO('yolov8x.pt')

In [ ]:
results = model('road_image.jpg')
#결과 시각화:
img = cv2.imread('road_image.jpg')
for r in results:
    annotated_img = r.plot()
cv2_imshow(annotated_img)

아직 인식 못하는 것들을 데이터 크롤링으로 데이터를 모아보자.

필요한 라이브러리 설치

In [ ]:
!pip install requests beautifulsoup4 Pillow


"속도계","횡단보도", "버스정류장", "도로 표지판", "교통 안전 시설물"] 데이터 모으기

크롤링 코드 작성

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from PIL import Image
from io import BytesIO
from google.colab import files


search_terms = ["속도계", "횡단보도", "버스정류장",  "도로 표지판", "교통 안전 시설물"]

for search_term in search_terms:
    # 이미지를 저장할 폴더 생성
    if not os.path.exists(search_term):
        os.makedirs(search_term)

    # Google 이미지 검색 URL
    url = f"https://www.google.com/search?q={search_term}&tbm=isch"

    # 웹 페이지 가져오기
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 이미지 태그 찾기
    img_tags = soup.find_all('img')

    # 이미지 다운로드
    for i, img in enumerate(img_tags):
        try:
            img_url = img['src']
            if img_url.startswith('http'):
                img_data = requests.get(img_url).content
                img = Image.open(BytesIO(img_data))
                img.save(f"{search_term}/{search_term}_{i}.jpg")
                print(f"Downloaded: {search_term}_{i}.jpg")
        except:
            print(f"Error downloading image {i}")

    print(f"Crawling completed for {search_term}!")

    # 다운로드한 이미지들을 zip 파일로 압축
    !zip -r {search_term}.zip {search_term}

    # zip 파일 다운로드
    files.download(f"{search_term}.zip")


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from PIL import Image
from io import BytesIO
from google.colab import files

# 검색어 설정
search_terms = ["속도계", "횡단보도", "버스정류장", "도로 경계석", "도로 표지판", "교통 안전 시설물"]

# 이미지를 저장할 폴더 생성
if not os.path.exists(search_term):
    os.makedirs(search_term)

# Google 이미지 검색 URL
url = f"https://www.google.com/search?q={search_term}&tbm=isch"

# 웹 페이지 가져오기
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 이미지 태그 찾기
img_tags = soup.find_all('img')

# 이미지 다운로드
for i, img in enumerate(img_tags):
    try:
        img_url = img['src']
        if img_url.startswith('http'):
            img_data = requests.get(img_url).content
            img = Image.open(BytesIO(img_data))
            img.save(f"{search_term}/{search_term}_{i}.jpg")
            print(f"Downloaded: {search_term}_{i}.jpg")
    except:
        print(f"Error downloading image {i}")

print("Crawling completed!")

# 다운로드한 이미지들을 zip 파일로 압축
!zip -r {search_term}.zip {search_term}

# zip 파일 다운로드
files.download(f"{search_term}.zip")
